In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
data = pd.read_parquet(".." + os.sep + "data" + os.sep +
                "dati_presenze_politici.parquet")


trasformazione delle date in oggetti datetime

In [3]:
data['DATA'] = data['DATA'].apply(lambda x: x.replace(" 00:00",""))
data['DATA'] = data['DATA'].apply(lambda x: x.replace(":00",""))
data.DATA = pd.to_datetime(data["DATA"],  format="%d.%m.%Y")


In [21]:
data.head(4)

,CANALE,PROGRAMMA,DATA,COGNOME,NOME,MICRO_CATEGORIA,ARGOMENTO,DURATA,TIPO_TEMPO
2,CANALE 5,TG5,2022-01-01,Berlusconi,Silvio,PDL - Forza Italia,Politica e attività istituzionali,42,Notizia
3,CANALE 5,TG5,2022-01-01,Calenda,Carlo,Azione - +Europa - Radicali Italiani,Politica e attività istituzionali,6,Notizia
4,CANALE 5,TG5,2022-01-01,Conte,Giuseppe,Movimento 5 Stelle,Politica e attività istituzionali,36,Notizia
5,CANALE 5,TG5,2022-01-01,D'Amato,Alessio,Partito Democratico,Cronaca,3,Notizia


filtro per soli politici

In [4]:
dati_politici = data[data.NOME != "Soggettivo Collettivo"]
dati_politici = dati_politici[dati_politici.NOME != "Soggetto Collettivo"]

creazione campo NOME_COGNOME

In [5]:
dati_politici['NOME_COGNOME'] = dati_politici['NOME'] + \
    " " + dati_politici['COGNOME']


calcolo presenze per politico

In [6]:
presenze = dati_politici.pivot_table(index="NOME_COGNOME",
                             values="DURATA", aggfunc=np.sum)#

In [7]:
presenze.reset_index(inplace=True)


In [8]:
presenze.sort_values(by="DURATA", ascending=False, inplace=True)

In [9]:
totale_politici = len(dati_politici.NOME_COGNOME.unique())


In [10]:
totale_politici


8547

ricerca più presenti nell'ultimo decile

In [11]:
#ultimo decile
p = presenze.DURATA.quantile(0.9)

In [12]:
presenze_politici_rilevanti = presenze[(presenze.DURATA > presenze.DURATA.quantile(0.9)) & (presenze.DURATA <= presenze.DURATA.max())]


di questi prendiamo l'ultimo decile

In [13]:
presenze_politici_top_rilevanti = presenze[(presenze.DURATA > presenze_politici_rilevanti.DURATA.quantile(
    0.9)) & (presenze.DURATA <= presenze.DURATA.max())]


In [14]:
presenze_politici_top_rilevanti.shape[0]

86

In [15]:
presenze_politici_top_rilevanti

,NOME_COGNOME,DURATA
5887,Matteo Salvini,4830594
7830,Silvio Berlusconi,2542977
5001,Luigi Di Maio,2460683
5880,Matteo Renzi,1896863
3719,Giorgia Meloni,1835590
...,...,...
6929,Pietro Grasso,167791
5412,Maria Elena Boschi,165889
3641,Gianni Cuperlo,165194
4933,Luciano Nobili,160188


In [22]:
data[['DATA','DURATA','TIPO_TEMPO']].groupby(['DATA']).sum()

,DURATA
DATA,
2018-01-01,13947
2018-01-02,22521
2018-01-03,34790
2018-01-04,37318
2018-01-05,31416
...,...
2022-06-26,13444
2022-06-27,51089
2022-06-28,40931


In [23]:
data.pivot_table(index="DATA", values="DURATA", aggfunc=np.sum)


,DURATA
DATA,
2018-01-01,13947
2018-01-02,22521
2018-01-03,34790
2018-01-04,37318
2018-01-05,31416
...,...
2022-06-26,13444
2022-06-27,51089
2022-06-28,40931


In [28]:
volume_notizie_parole = data.groupby(['DATA', 'TIPO_TEMPO'])['DURATA'].aggregate('sum').unstack().reset_index()

In [34]:
volume_notizie_parole.sort_values(by="DATA", ascending=True, inplace=True)

In [49]:
volume_notizie_parole


TIPO_TEMPO,DATA,Notizia,Parola
0,2018-01-01,8543,5404
1,2018-01-02,13613,8908
2,2018-01-03,18222,16568
3,2018-01-04,17561,19757
4,2018-01-05,16868,14548
...,...,...,...
1637,2022-06-26,3858,9586
1638,2022-06-27,15647,35442
1639,2022-06-28,12438,28493
1640,2022-06-29,8713,31095


In [50]:
for idx,row in volume_notizie_parole.iterrows():
    day = row.DATA.strftime('%Y/%m/%d')
    parole = row.Parola
    notize = row.Notizia
    print("['%s', %s, 'INTERVENTI']," % (day, parole))
    print("['%s', %s, 'NOTIZIE']," % (day, notize))


['2018/01/01', 5404, 'INTERVENTI'],
['2018/01/01', 8543, 'NOTIZIE'],
['2018/01/02', 8908, 'INTERVENTI'],
['2018/01/02', 13613, 'NOTIZIE'],
['2018/01/03', 16568, 'INTERVENTI'],
['2018/01/03', 18222, 'NOTIZIE'],
['2018/01/04', 19757, 'INTERVENTI'],
['2018/01/04', 17561, 'NOTIZIE'],
['2018/01/05', 14548, 'INTERVENTI'],
['2018/01/05', 16868, 'NOTIZIE'],
['2018/01/06', 9367, 'INTERVENTI'],
['2018/01/06', 13374, 'NOTIZIE'],
['2018/01/07', 19509, 'INTERVENTI'],
['2018/01/07', 28282, 'NOTIZIE'],
['2018/01/08', 31860, 'INTERVENTI'],
['2018/01/08', 29052, 'NOTIZIE'],
['2018/01/09', 47921, 'INTERVENTI'],
['2018/01/09', 24635, 'NOTIZIE'],
['2018/01/10', 56296, 'INTERVENTI'],
['2018/01/10', 24110, 'NOTIZIE'],
['2018/01/11', 55133, 'INTERVENTI'],
['2018/01/11', 28002, 'NOTIZIE'],
['2018/01/12', 47423, 'INTERVENTI'],
['2018/01/12', 32099, 'NOTIZIE'],
['2018/01/13', 36848, 'INTERVENTI'],
['2018/01/13', 31784, 'NOTIZIE'],
['2018/01/14', 42286, 'INTERVENTI'],
['2018/01/14', 35144, 'NOTIZIE'],
['2018/01/